<a href="https://colab.research.google.com/github/jsgf04/TdR_Physics_behind_skiing/blob/main/Simuladors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Simulador del moviment del pèndol centrífug invertit [Nivell: principiant-intermedi]**




Variables que et demanarà:


*   **Alçada de l'esquiador (m):** per calcular la posició aproximada del CM (centre de massa)
*   **Radi dels esquís (m)**
*   **Velocitat de l'esquiador durant el trajecte (m/s)(constant)**
*   **Temps final o durada del trajecte (s)**
*   **Angle inicial del pèndol/de l'esquiador (º):**

Hi han **dos** casos:

-Quan l'esquiador/el pèndol té 3 posicions d'equilibri. Dues de simètriques (t'ho indica abans que intodueixis l'angle) i una de vertical (0º). Òbviament, si aquest límit és sobrepassat l'esquiador perd l'equilibi i cau (en el cas del pèndol actua estranyament). Per això, diem que l'inclinació del pènol/de l'esquiador no pot passar del límit o angle màxim (99% de l'angle d'equilibri).

-Quan l'esquiador/el pèndol té una posició d'equilibri que és la vertical (0º) i, a més a més, si tries qualsevol angle, mentre sigui realista o no sigui extrem, pot estar oscil·lant constantment sense quedar-se en cap posició d'equilibri.

***Posició d'equilibri significa la posició de l'esquiador en què la velocitat angular del pèndol és 0.**


---
Normes per al correcte funcionament del simulador:

-Nombres decimals amb '  .  ' , no amb '  ,  '  .

**Correcte :** 12.7

**Incorrecte :** 12,7


---
Per iniciar la simulació cliqueu el botó de "play" al racó superior esquerre del codi o bé feu la combinació de tecles 

```
Ctrl+Enter
```




In [ ]:
import numpy as np
from numpy import sin,cos,sign,sqrt,arcsin,pi, arctan, tan
from scipy.integrate import solve_ivp
from matplotlib import pyplot as plt
from termcolor import colored
from matplotlib.pyplot import figure
import sympy as smp
from matplotlib.animation import FuncAnimation, PillowWriter
from matplotlib import animation, rc
from IPython.display import HTML
 
print(colored("ADVERTÈNCIA: EL SIMULADOR ÉS VÀLID FINS APROXIMADAMENT 65º-70º","red"))
g = 9.807
height = float(input("Alçada de l'esquiador (m): "))
l = 0.56*height
R_sc = float(input("Radi dels esquís (m): "))
V = float(input("Velocitat de l'esquiador durant el trajecte (m/s): "))
 
if V**2/(g*R_sc)<1:
  Psi_eq = round(np.arcsin(V**2/(g*R_sc))*(180/np.pi), 3)
else:
  Psi_eq = "No hi han posicions d'equilibri"
 
if V**2/(g*R_sc)<1:
  text2 = "${\Psi}_{equilibri}$ = ±"
else:
  text2 = "${\Psi}_{equilibri}$ = "
 
def equations(t,y):
    dy=np.zeros([2])
    dy[0] = y[1]
    dy[1] = (g/l)*np.sin(y[0])-((V**2)/(l*R_sc))*np.sign(y[0])
 
    return dy
 
final_time = int(input("Temps final o durada del trajecte (s): "))
interval=30*final_time
time = np.linspace(0, final_time, interval)
 
if V**2/(g*R_sc)<1:
  Psi_equilibri_arr = round(0.98*Psi_eq,3)
  print("Angle d'equilibri per referència: ±"+str(Psi_eq)+"º",  colored(("[NO POT PASSAR DE ± 0.99 · \u03A8  ≈ ± "+str(Psi_equilibri_arr)+"... º]"),"red"))
else:
  print("No hi han posicions d'equilibri, és a dir, el pèndol/l'esquiador oscil·larà constantment a qualsevol angle sempre i quan no sigui extrem (70º-90º, 90º seria inclinar-se paral·lelament amb el terra | A partir de 70º l'esquiador pateix fins a 3 g s de força en cas el cas que la pista sigui horitzontal, això requereix molta força muscular.)")
 
initial_Psi = float(input("Angle inicial del pèndol/esquiador respecte de la vertical, \u03A6 (º): "))
y0 = [np.radians(initial_Psi), np.radians(0.0)]
 
res = solve_ivp(equations, (0,final_time), y0, method='RK45', t_eval=time, dense_output=True, rtol=1e-8, atol=1e-10)
 
t, psi, omega = res.t, res.y[0], res.y[1]
 
pos_y= l*np.sin(psi)
pos_z= l*np.cos(psi)
 
fig = plt.figure()
ax = plt.axes()
gca = plt.gca()
gca.set_aspect("equal", adjustable="box")
plt.vlines(x=0,ymin=0, ymax=2,colors="firebrick",linestyles="--",linewidth=1, zorder=0)
plt.hlines(y=0,xmin=-3, xmax=3,colors="black",linewidth=1,zorder=0)
plt.scatter(100,100,26,c="red", label="CM",zorder=2)
plt.xlabel("X [m]", fontsize=9, fontweight = "bold")
plt.ylabel("Z [m]", fontsize=9, fontweight = "bold")
plt.title('Moviment del pèndol centrífug invertit ')
gca.axes.xaxis.set_ticks([])
gca.axes.yaxis.set_ticks([l])
plt.ylim(-0.25,l+0.25)
plt.xlim(-l-0.25,l+0.25)
plt.legend(loc = "upper right", fontsize=8)
trans = plt.axes().transAxes
time_text1 = ax.text(0.01, 0.93, '', transform=trans)
psi_text = ax.text(0.02, 0.85, '', transform=trans)
line, = plt.plot([], [], 'k', lw=0.4, zorder=1)
skis_frontal, = plt.plot([], [], 'k', lw=2, zorder=1)
point, = plt.plot([],[], marker="o", c="red")
plt.close()

skis_Y_ini = -0.07*np.cos(psi)
skis_Y_fin = 0.07*np.cos(psi)
skis_Z_ini = 0.07*np.sin(psi)
skis_Z_fin = -0.07*np.sin(psi)

def init():
  line.set_data([], [])
  skis_frontal.set_data([], [])
  point.set_data([],[])
  time_text1.set_text('')
  psi_text.set_text('')
  return line, skis_frontal, point, time_text1, psi_text

def animate(i):
  line.set_data([0,pos_y[i]], [0,pos_z[i]])
  skis_frontal.set_data([skis_Y_ini[i], skis_Y_fin[i]], [skis_Z_ini[i], skis_Z_fin[i]])
  point.set_data([pos_y[i]],[pos_z[i]])
  time_text1.set_text('Temps = %.1f s' %(i/30))
  psi_text.set_text('\u03A6 = \u03A8 = %.2f º' %np.degrees(psi[i]))
  return line, skis_frontal, point, time_text1, psi_text
 
anim = FuncAnimation(fig, animate,init_func=init, frames=interval, interval=33.3333334, blit=True)
 
if V**2/(g*R_sc)<1 and (initial_Psi > Psi_eq or initial_Psi < -Psi_eq):
  adv1 = "(L'esquiador ha perdut l'equilibri i ha caigut)"
else:
  adv1 = " "
 
print(colored(adv1,"red",attrs=['bold']))
HTML(anim.to_html5_video())

ADVERTÈNCIA: EL SIMULADOR ÉS VÀLID FINS APROXIMADAMENT 65º-70º
Alçada de l'esquiador (m): 1.8
Radi dels esquís (m): 14
Velocitat de l'esquiador durant el trajecte (m/s): 12
Temps final o durada del trajecte (s): 10
No hi han posicions d'equilibri, és a dir, el pèndol/l'esquiador oscil·larà constantment a qualsevol angle sempre i quan no sigui extrem (70º-90º, 90º seria inclinar-se paral·lelament amb el terra | A partir de 70º l'esquiador pateix fins a 3 g s de força en cas el cas que la pista sigui horitzontal, això requereix molta força muscular.)
Angle inicial del pèndol/esquiador respecte de la vertical, Φ (º): 45


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:74: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.


#**Simulador del moviment del pèndol centrífug invertit amb flexió i extensió de cames [Nivell: experimentat]**



Variables que et demanarà:

*   **Alçada de l'esquiador (m):** per calcular la posició aproximada del CM (centre de massa)
*   **Radi dels esquís (m)**
*   **Velocitat de l'esquiador durant el trajecte (m/s)(constant)**
*   **Temps final o durada del trajecte (s)**
*   **Angle inicial del pèndol/de l'esquiador (º):** 

Hi han **dos** casos:

-Quan l'esquiador/el pèndol té 3 posicions d'equilibri. Dues de simètriques (t'ho indica abans que intodueixis l'angle) i una de vertical (0º). Òbviament, si aquest límit és sobrepassat l'esquiador perd l'equilibi i cau (en el cas del pèndol actua estranyament). Per això, diem que l'inclinació del pènol/de l'esquiador no pot passar del límit o angle màxim (99% de l'angle d'equilibri).

-Quan l'esquiador/el pèndol té una posició d'equilibri que és la vertical (0º) i, a més a més, si tries qualsevol angle, mentre sigui realista o no sigui extrem, pot estar oscil·lant constantment sense quedar-se en cap posició d'equilibri.

***Posició d'equilibri significa la posició de l'esquiador en què la velocitat angular del pèndol és 0.**

*   **Percentatge de flexió i extensió de les cames:** quan l'esquiador està en posició vertical les cames estan flexionades, per altra banda, si està al extrem de la curva la cama exterior està extesa (l'altra és paral·lela a l'exterior i, per tant, sempre està flexionada, vertical o no vertical). Generalment, com més petit és el percentatge, menys es nota la flexió i extensió de cames, i, a l'inrevés, com més s'aproxima al 100% més es nota la flexió i extensió de cames.



---
Normes per al correcte funcionament del simulador:

-Nombres decimals amb '  .  ' , no amb '  ,  '  .

**Correcte :** 12.7

**Incorrecte :** 12,7


---
Per iniciar la simulació cliqueu el botó de "play" al racó superior esquerre del codi o bé feu la combinació de tecles

```
Ctrl+Enter
```




In [12]:
import numpy as np
from numpy import sin,cos,sign,sqrt,arcsin,pi, arctan, tan
from scipy.integrate import solve_ivp
from matplotlib import pyplot as plt
from termcolor import colored
from matplotlib.pyplot import figure
import sympy as smp
from matplotlib.animation import FuncAnimation, PillowWriter
from matplotlib import animation, rc
from IPython.display import HTML

print(colored("ADVERTÈNCIA: EL SIMULADOR ÉS VÀLID FINS APROXIMADAMENT 65º-70º","red"))
g = 9.807
height = float(input("Alçada de l'esquiador (m): "))
l_o = 0.56*height
R_sc = float(input("Radi dels esquís (m): "))
V = float(input("Velocitat de l'esquiador durant el trajecte (m/s): "))
 
if V**2/(g*R_sc)<1:
  Psi_eq = round(np.arcsin(V**2/(g*R_sc))*(180/np.pi), 3)
else:
  Psi_eq = "No hi han posicions d'equilibri"
 
if V**2/(g*R_sc)<1:
  text2 = "${\Psi}_{equilibri}$ = ±"
else:
  text2 = "${\Psi}_{equilibri}$ = "
 
def equations(t,y):
    dy=np.zeros([2])
    dy[0] = y[1]
    dy[1] = (g/(l_o*(1-b*np.cos(y[0]))**(1./3)))*np.sin(y[0])-((V**2)/(R_sc*l_o*(1-b*np.cos(y[0]))**(1./3)))*np.sign(y[0]) - (2 * b * (y[1])**2 * np.sin(y[0]))/(3-3 * b * np.cos(y[0]))
 
    return dy
 
final_time = int(input("Temps final o durada del trajecte (s): "))
interval=30*final_time
time = np.linspace(0, final_time, interval)
 
if V**2/(g*R_sc)<1:
  Psi_equilibri_arr = round(0.98*Psi_eq,3)
  print("Angle d'equilibri per referència: ±"+str(Psi_eq)+"º",  colored(("[NO POT PASSAR DE ± 0.99 · \u03A8  ≈ ± "+str(Psi_equilibri_arr)+"... º]"),"red"))
else:
  print("No hi han posicions d'equilibri, és a dir, el pèndol/l'esquiador oscil·larà constantment a qualsevol angle sempre i quan no sigui extrem (70º-90º, 90º seria inclinar-se paral·lelament amb el terra | A partir de 70º l'esquiador pateix fins a 3 g s de força en cas el cas que la pista sigui horitzontal, això requereix molta força muscular.)")
 
initial_Psi = float(input("Angle inicial del pèndol/esquiador respecte de la vertical, \u03A6 (º): "))
P = float(input("Percentatge de flexió i extensió de les cames (0-100%): "))
b = P/100
y0 = [np.radians(initial_Psi), np.radians(0.0)]
 

res = solve_ivp(equations, (0,final_time), y0, method='RK45', t_eval=time, dense_output=True, rtol=1e-8, atol=1e-10)
 
t, psi, omega = res.t, res.y[0], res.y[1]
 
pos_y= l_o*np.sin(psi)*(1-b*np.cos(psi))**(1./3)
pos_z= l_o*np.cos(psi)*(1-b*np.cos(psi))**(1./3)
 
fig = plt.figure()
ax = plt.axes()
gca = plt.gca()
gca.set_aspect("equal", adjustable="box")
plt.vlines(x=0,ymin=0, ymax=2,colors="firebrick",linestyles="--",linewidth=1, zorder=0)
plt.hlines(y=0,xmin=-3, xmax=3,colors="black",linewidth=1,zorder=0)
plt.scatter(100,100,26,c="red", label="CM",zorder=2)
plt.xlabel("X [m]", fontsize=9, fontweight = "bold")
plt.ylabel("Z [m]", fontsize=9, fontweight = "bold")
plt.title('Moviment del pèndol centrífug invertit ')
gca.axes.xaxis.set_ticks([])
gca.axes.yaxis.set_ticks([l_o*(1-b)**(1./3)])
plt.ylim(-0.25,l_o+0.25)
plt.xlim(-l_o-0.25,l_o+0.25)
plt.legend(loc = "upper right", fontsize=8)
trans = plt.axes().transAxes
time_text1 = ax.text(0.01, 0.93, '', transform=trans)
psi_text = ax.text(0.02, 0.85, '', transform=trans)
length_text = ax.text(0.02, 0.77, '', transform=trans)
line, = plt.plot([], [], 'k', lw=0.4, zorder=1)
skis_frontal, = plt.plot([], [], 'k', lw=2, zorder=1)
point, = plt.plot([],[], marker="o", c="red")
plt.close()

skis_Y_ini = -0.07*np.cos(psi)
skis_Y_fin = 0.07*np.cos(psi)
skis_Z_ini = 0.07*np.sin(psi)
skis_Z_fin = -0.07*np.sin(psi)
l = l_o * (1-b*np.cos(psi))**(1./3)

def init():
  line.set_data([], [])
  skis_frontal.set_data([], [])
  point.set_data([],[])
  time_text1.set_text('')
  psi_text.set_text('')
  length_text.set_text('')
  return line, skis_frontal, point, time_text1, psi_text, length_text

def animate(i):
  line.set_data([0,pos_y[i]], [0,pos_z[i]])
  skis_frontal.set_data([skis_Y_ini[i], skis_Y_fin[i]], [skis_Z_ini[i], skis_Z_fin[i]])
  point.set_data([pos_y[i]],[pos_z[i]])
  time_text1.set_text('Temps = %.1f s' %(i/30))
  psi_text.set_text('\u03A6 = \u03A8 = %.2f º' %np.degrees(psi[i]))
  length_text.set_text('l = %.3f m' %l[i])
  return line, skis_frontal, point, time_text1, psi_text, length_text
 
anim = FuncAnimation(fig, animate,init_func=init, frames=interval, interval=33.3333334, blit=True)
 
if V**2/(g*R_sc)<1 and (initial_Psi > Psi_eq or initial_Psi < -Psi_eq):
  adv1 = "(L'esquiador ha perdut l'equilibri i ha caigut)"
else:
  adv1 = " "
 
print(colored(adv1,"red",attrs=['bold']))
HTML(anim.to_html5_video())

ADVERTÈNCIA: EL SIMULADOR ÉS VÀLID FINS APROXIMADAMENT 65º-70º
Alçada de l'esquiador (m): 1.8
Radi dels esquís (m): 14
Velocitat de l'esquiador durant el trajecte (m/s): 12
Temps final o durada del trajecte (s): 10
No hi han posicions d'equilibri, és a dir, el pèndol/l'esquiador oscil·larà constantment a qualsevol angle sempre i quan no sigui extrem (70º-90º, 90º seria inclinar-se paral·lelament amb el terra | A partir de 70º l'esquiador pateix fins a 3 g s de força en cas el cas que la pista sigui horitzontal, això requereix molta força muscular.)
Angle inicial del pèndol/esquiador respecte de la vertical, Φ (º): 45
Percentatge de flexió i extensió de les cames (0-100%): 80


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:74: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.


#**Simulador del moviment del pèndol centrífug invertit amb angulació [Nivell: expert]**



Variables que et demanarà:

*   **Alçada de l'esquiador (m):** per calcular la posició aproximada del CM (centre de massa)
*   **Radi dels esquís (m)**
*  **Velocitat de l'esquiador durant el trajecte (m/s)(constant)**
*   **Temps final o durada del trajecte (s)**
*  **Angle inicial del pèndol/de l'esquiador (º):**

Hi han **dos** casos:

-Quan l'esquiador/el pèndol té 3 posicions d'equilibri. Dues de simètriques (t'ho indica abans que intodueixis l'angle) i una de vertical (0º). Òbviament, si aquest límit és sobrepassat l'esquiador perd l'equilibi i cau (en el cas del pèndol actua estranyament). Per això, diem que l'inclinació del pènol/de l'esquiador no pot passar del límit o angle màxim (99% de l'angle d'equilibri).

-Quan l'esquiador/el pèndol té una posició d'equilibri que és la vertical (0º) i, a més a més, si tries qualsevol angle, mentre sigui realista o no sigui extrem, pot estar oscil·lant constantment sense quedar-se en cap posició d'equilibri.

***Posició d'equilibri significa la posició de l'esquiador en què la velocitat angular del pèndol és 0.**


* **Percentatge d'angulació (Angle d'amplitud del pèndol = Percentatge * Angle de l'esquí respecte del terra):** l'angle de l'esquí sempre serà més gran que la de l'esquiador. Està comprès entre 0-100%. A 100% no hi ha angulació, atès que és com si multipliquessis per 1.)

---
Normes per al correcte funcionament del simulador:

-Nombres decimals amb '  .  ' , no amb '  ,  '  .

**Correcte :** 12.7

**Incorrecte :** 12,7


---
Per iniciar la simulació cliqueu el botó de "play" al racó superior esquerre del codi o bé feu la combinació de tecles 

```
Ctrl+Enter
```

In [ ]:
import numpy as np
from numpy import sin,cos,sign,sqrt,arcsin,pi, arctan, tan
from scipy.integrate import solve_ivp
from matplotlib import pyplot as plt
from termcolor import colored
from matplotlib.pyplot import figure
import sympy as smp
from matplotlib.animation import FuncAnimation, PillowWriter
from matplotlib import animation, rc
from IPython.display import HTML
 
print(colored("ADVERTÈNCIA: EL SIMULADOR ÉS VÀLID FINS APROXIMADAMENT 65º-70º","red"))
g = 9.807
height = float(input("Alçada de l'esquiador (m): "))
l = 0.56*height
R_sc = float(input("Radi dels esquís (m): "))
V = float(input("Velocitat de l'esquiador durant el trajecte (m/s): "))
 
if V**2/(g*R_sc)<1:
  Psi_eq = round(np.arcsin(V**2/(g*R_sc))*(180/np.pi), 3)
else:
  Psi_eq = "No hi han posicions d'equilibri"
 
if V**2/(g*R_sc)<1:
  text2 = "${\Psi}_{equilibri}$ = ±"
else:
  text2 = "${\Psi}_{equilibri}$ = "
 
def equations(t,y):
    dy=np.zeros([2])
    dy[0] = y[1]
    dy[1] = (g/l*A)*np.sin(A*y[0])-(V**2 * np.cos(A*y[0]) * np.sign(y[0]))/(l*R_sc*A*np.cos(y[0]))
 
    return dy
 
final_time = int(input("Temps final o durada del trajecte (s): "))
interval=30*final_time
time = np.linspace(0, final_time, interval)
 
if V**2/(g*R_sc)<1:
  Psi_equilibri_arr = round(0.98*Psi_eq,3)
  print("Angle d'equilibri per referència: ±"+str(Psi_eq)+"º",  colored(("[NO POT PASSAR DE ± 0.99 · \u03A8  ≈ ± "+str(Psi_equilibri_arr)+"... º]"),"red"))
else:
  print("No hi han posicions d'equilibri, és a dir, el pèndol/l'esquiador oscil·larà constantment a qualsevol angle sempre i quan no sigui extrem (70º-90º, 90º seria inclinar-se paral·lelament amb el terra | A partir de 70º l'esquiador pateix fins a 3 g s de força en cas el cas que la pista sigui horitzontal, això requereix molta força muscular.)")
 
initial_Psi = float(input("Angle inicial del pèndol/esquiador respecte de la vertical, \u03A6 (º): "))
print("Mínim percentatge d'angulació (l'esquí assoleix 90º): "+str((initial_Psi*100)/90)+" % (es recomana que sigui una mica més alt encara per evitar errors)")
print("Màxim percentatge d'angulació (no hi ha angulació): 100 %")
P = float(input("Percentatge d'angulació: "))
A = P/100
y0 = [np.radians(initial_Psi/A), np.radians(0.0)]
 
res = solve_ivp(equations, (0,final_time), y0, method='RK45', t_eval=time, dense_output=True, rtol=1e-8, atol=1e-10)
 
t, psi, omega = res.t, res.y[0], res.y[1]
 
pos_x= l*np.sin(A*psi)
pos_z= l*np.cos(A*psi)
 
fig = plt.figure()
ax = plt.axes()
gca = plt.gca()
gca.set_aspect("equal", adjustable="box")
plt.vlines(x=0,ymin=0, ymax=2,colors="firebrick",linestyles="--",linewidth=1, zorder=0)
plt.hlines(y=0,xmin=-3, xmax=3,colors="black",linewidth=1,zorder=0)
plt.scatter(100,100,26,c="red", label="CM",zorder=2)
plt.xlabel("X [m]", fontsize=9, fontweight = "bold")
plt.ylabel("Z [m]", fontsize=9, fontweight = "bold")
plt.title('Moviment del pèndol centrífug invertit ')
gca.axes.xaxis.set_ticks([])
gca.axes.yaxis.set_ticks([l])
plt.ylim(-0.25,l+0.25)
plt.xlim(-l-0.25,l+0.25)
plt.legend(loc = "upper right", fontsize=8)
trans = plt.axes().transAxes
time_text1 = ax.text(0.01, 0.93, '', transform=trans)
psi_text = ax.text(0.02, 0.77, '', transform=trans)
phi_text = ax.text(0.02, 0.85, '', transform=trans)
line, = plt.plot([], [], 'k', lw=0.4, zorder=1)
skis_frontal, = plt.plot([], [], 'k', lw=2, zorder=1)
point, = plt.plot([],[], marker="o", c="red")
plt.close()

skis_Y_ini = -0.07*np.cos(psi)
skis_Y_fin = 0.07*np.cos(psi)
skis_Z_ini = 0.07*np.sin(psi)
skis_Z_fin = -0.07*np.sin(psi)

def init():
  line.set_data([], [])
  skis_frontal.set_data([], [])
  point.set_data([],[])
  time_text1.set_text('')
  psi_text.set_text('')
  phi_text.set_text('')
  return line, skis_frontal, point, time_text1, psi_text, phi_text

def animate(i):
  line.set_data([0,pos_x[i]], [0,pos_z[i]])
  skis_frontal.set_data([skis_Y_ini[i], skis_Y_fin[i]], [skis_Z_ini[i], skis_Z_fin[i]])
  point.set_data([pos_x[i]],[pos_z[i]])
  time_text1.set_text('Temps = %.1f s' %(i/30))
  psi_text.set_text('\u03A8 = %.2f º' %np.degrees(psi[i]))
  phi_text.set_text('\u03A6 = %.2f º' %np.degrees(A*psi[i]))
  return line, skis_frontal, point, time_text1, psi_text, phi_text
 
anim = FuncAnimation(fig, animate,init_func=init, frames=interval, interval=33.3333334, blit=True)
 
if V**2/(g*R_sc)<1 and (initial_Psi > Psi_eq or initial_Psi < -Psi_eq):
  adv1 = "(L'esquiador ha perdut l'equilibri i ha caigut)"
else:
  adv1 = " "
 
print(colored(adv1,"red",attrs=['bold']))
HTML(anim.to_html5_video())

ADVERTÈNCIA: EL SIMULADOR ÉS VÀLID FINS APROXIMADAMENT 65º-70º
Alçada de l'esquiador (m): 1.8
Radi dels esquís (m): 14
Velocitat de l'esquiador durant el trajecte (m/s): 12
Temps final o durada del trajecte (s): 10
No hi han posicions d'equilibri, és a dir, el pèndol/l'esquiador oscil·larà constantment a qualsevol angle sempre i quan no sigui extrem (70º-90º, 90º seria inclinar-se paral·lelament amb el terra | A partir de 70º l'esquiador pateix fins a 3 g s de força en cas el cas que la pista sigui horitzontal, això requereix molta força muscular.)
Angle inicial del pèndol/esquiador respecte de la vertical, Φ (º): 60
Mínim percentatge d'angulació (l'esquí assoleix 90º): 66.66666666666667 % (es recomana que sigui una mica més alt encara per evitar errors)
Màxim percentatge d'angulació (no hi ha angulació): 100 %
Percentatge d'angulació: 75


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:78: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.


#**Simulador del moviment del pèndol centrífug invertit amb flexió i extensió de cames i amb angulació afegida [Nivell: atleta-professional]**



Variables que et demanarà:

*   **Alçada de l'esquiador (m):** per calcular la posició aproximada del CM (centre de massa)
*   **Radi dels esquís (m)**
*  **Velocitat de l'esquiador durant el trajecte (m/s)(constant)**
*   **Temps final o durada del trajecte (s)**
*  **Angle inicial del pèndol/de l'esquiador (º):**

Hi han **dos** casos:

-Quan l'esquiador/el pèndol té 3 posicions d'equilibri. Dues de simètriques (t'ho indica abans que intodueixis l'angle) i una de vertical (0º). Òbviament, si aquest límit és sobrepassat l'esquiador perd l'equilibi i cau (en el cas del pèndol actua estranyament). Per això, diem que l'inclinació del pènol/de l'esquiador no pot passar del límit o angle màxim (99% de l'angle d'equilibri).

-Quan l'esquiador/el pèndol té una posició d'equilibri que és la vertical (0º) i, a més a més, si tries qualsevol angle, mentre sigui realista o no sigui extrem, pot estar oscil·lant constantment sense quedar-se en cap posició d'equilibri.

***Posició d'equilibri significa la posició de l'esquiador en què la velocitat angular del pèndol és 0.**



*   **Percentatge de flexió i extensió de les cames:** quan l'esquiador està en posició vertical les cames estan flexionades, per altra banda, si està al extrem de la curva la cama exterior està extesa (l'altra és paral·lela a l'exterior i, per tant, sempre està flexionada, vertical o no vertical). Generalment, com més petit és el percentatge, menys es nota la flexió i extensió de cames, i, a l'inrevés, com més s'aproxima al 100% més es nota la flexió i extensió de cames.


* **Percentatge d'angulació (Angle d'amplitud del pèndol = Percentatge * Angle de l'esquí respecte del terra):** l'angle de l'esquí sempre serà més gran que la de l'esquiador. Està comprès entre 0-100%. A 100% no hi ha angulació, atès que és com si multipliquessis per 1.)

---
Normes per al correcte funcionament del simulador:

-Nombres decimals amb '  .  ' , no amb '  ,  '  .

**Correcte :** 12.7

**Incorrecte :** 12,7


---
Per iniciar la simulació cliqueu el botó de "play" al racó superior esquerre del codi o bé feu la combinació de tecles

```
Ctrl+Enter
```

In [11]:
import numpy as np
from numpy import sin,cos,sign,sqrt,arcsin,pi, arctan, tan
from scipy.integrate import solve_ivp
from matplotlib import pyplot as plt
from termcolor import colored
from matplotlib.pyplot import figure
import sympy as smp
from matplotlib.animation import FuncAnimation, PillowWriter
from matplotlib import animation, rc
from IPython.display import HTML

print(colored("ADVERTÈNCIA: EL SIMULADOR ÉS VÀLID FINS APROXIMADAMENT 65º-70º","red"))
g = 9.807
height = float(input("Alçada de l'esquiador (m): "))
l_o = 0.56*height
R_sc = float(input("Radi dels esquís (m): "))
V = float(input("Velocitat de l'esquiador durant el trajecte (m/s): "))
 
if V**2/(g*R_sc)<1:
  Psi_eq = round(np.arcsin(V**2/(g*R_sc))*(180/np.pi), 3)
else:
  Psi_eq = "No hi han posicions d'equilibri"
 
if V**2/(g*R_sc)<1:
  text2 = "${\Psi}_{equilibri}$ = ±"
else:
  text2 = "${\Psi}_{equilibri}$ = "
 
def equations(t,y):
    dy=np.zeros([2])
    dy[0] = y[1]
    dy[1] = (g/(A * l_o * (1-b * np.cos(A*y[0]))**(1./3)))*np.sin(A*y[0])-(V**2 * np.cos(A*y[0]) * np.sign(y[0]))/(l_o * (1-b * np.cos(A*y[0]))**(1./3) * R_sc * A * np.cos(y[0])) - (2 * b * (y[1])**2 * np.sin(A*y[0]))/(3*A-3* A * b * np.cos(A*y[0]))
 
    return dy
 
final_time = int(input("Temps final o durada del trajecte (s): "))
interval=30*final_time
time = np.linspace(0, final_time, interval)
 
if V**2/(g*R_sc)<1:
  Psi_equilibri_arr = round(0.98*Psi_eq,3)
  print("Angle d'equilibri per referència: ±"+str(Psi_eq)+"º",  colored(("[NO POT PASSAR DE ± 0.99 · \u03A8  ≈ ± "+str(Psi_equilibri_arr)+"... º]"),"red"))
else:
  print("No hi han posicions d'equilibri, és a dir, el pèndol/l'esquiador oscil·larà constantment a qualsevol angle sempre i quan no sigui extrem (70º-90º, 90º seria inclinar-se paral·lelament amb el terra | A partir de 70º l'esquiador pateix fins a 3 g s de força en cas el cas que la pista sigui horitzontal, això requereix molta força muscular.)")
 
initial_Psi = float(input("Angle inicial del pèndol/esquiador respecte de la vertical, \u03A6 (º): "))
P_1 = float(input("Percentatge de flexió i extensió de les cames (0-100%): "))
b = P_1/100
print("Mínim percentatge d'angulació (l'esquí assoleix 90º): "+str((initial_Psi*100)/90)+" % (es recomana que sigui una mica més alt encara per evitar errors)")
print("Màxim percentatge d'angulació (no hi ha angulació): 100 %")
P_2 = float(input("Percentatge d'angulació: "))
A = P_2/100
y0 = [np.radians(initial_Psi/A), np.radians(0.0)]
 

res = solve_ivp(equations, (0,final_time), y0, method='RK45', t_eval=time, dense_output=True, rtol=1e-8, atol=1e-10)
 
t, psi, omega = res.t, res.y[0], res.y[1]
 
fig = plt.figure()
ax = plt.axes()
gca = plt.gca()
gca.set_aspect("equal", adjustable="box")
plt.vlines(x=0,ymin=0, ymax=2,colors="firebrick",linestyles="--",linewidth=1, zorder=0)
plt.hlines(y=0,xmin=-3, xmax=3,colors="black",linewidth=1,zorder=0)
plt.scatter(100,100,26,c="red", label="CM",zorder=2)
plt.xlabel("X [m]", fontsize=9, fontweight = "bold")
plt.ylabel("Z [m]", fontsize=9, fontweight = "bold")
plt.title('Moviment del pèndol centrífug invertit ')
gca.axes.xaxis.set_ticks([])
gca.axes.yaxis.set_ticks([l_o*(1-b)**(1./3)])
plt.ylim(-0.25,l_o+0.25)
plt.xlim(-l_o-0.25,l_o+0.25)
plt.legend(loc = "upper right", fontsize=8)
trans = plt.axes().transAxes
time_text1 = ax.text(0.01, 0.93, '', transform=trans)
psi_text = ax.text(0.02, 0.85, '', transform=trans)
phi_text = ax.text(0.02, 0.77, '', transform=trans)
length_text = ax.text(0.02, 0.69, '', transform=trans)
line, = plt.plot([], [], 'k', lw=0.4, zorder=1)
skis_frontal, = plt.plot([], [], 'k', lw=2, zorder=1)
point, = plt.plot([],[], marker="o", c="red")
plt.close()

skis_Y_ini = -0.07*np.cos(psi)
skis_Y_fin = 0.07*np.cos(psi)
skis_Z_ini = 0.07*np.sin(psi)
skis_Z_fin = -0.07*np.sin(psi)
l = l_o * (1-b*np.cos(A*psi))**(1./3)
pos_y= l*np.sin(A*psi)
pos_z= l*np.cos(A*psi)

def init():
  line.set_data([], [])
  skis_frontal.set_data([], [])
  point.set_data([],[])
  time_text1.set_text('')
  psi_text.set_text('')
  phi_text.set_text('')
  length_text.set_text('')
  return line, skis_frontal, point, time_text1, psi_text, phi_text, length_text

def animate(i):
  line.set_data([0,pos_y[i]], [0,pos_z[i]])
  skis_frontal.set_data([skis_Y_ini[i], skis_Y_fin[i]], [skis_Z_ini[i], skis_Z_fin[i]])
  point.set_data([pos_y[i]],[pos_z[i]])
  time_text1.set_text('Temps = %.1f s' %(i/30))
  psi_text.set_text('\u03A8 = %.2f º' %np.degrees(psi[i]))
  phi_text.set_text('\u03A6 = %.2f º' %np.degrees(A*psi[i]))
  length_text.set_text('l = %.3f m' %l[i])
  return line, skis_frontal, point, time_text1, psi_text, phi_text, length_text
 
anim = FuncAnimation(fig, animate,init_func=init, frames=interval, interval=33.3333334, blit=True)
 
if V**2/(g*R_sc)<1 and (initial_Psi > Psi_eq or initial_Psi < -Psi_eq):
  adv1 = "(L'esquiador ha perdut l'equilibri i ha caigut)"
else:
  adv1 = " "
 
print(colored(adv1,"red",attrs=['bold']))
HTML(anim.to_html5_video())

ADVERTÈNCIA: EL SIMULADOR ÉS VÀLID FINS APROXIMADAMENT 65º-70º
Alçada de l'esquiador (m): 1.8
Radi dels esquís (m): 14
Velocitat de l'esquiador durant el trajecte (m/s): 12
Temps final o durada del trajecte (s): 10
No hi han posicions d'equilibri, és a dir, el pèndol/l'esquiador oscil·larà constantment a qualsevol angle sempre i quan no sigui extrem (70º-90º, 90º seria inclinar-se paral·lelament amb el terra | A partir de 70º l'esquiador pateix fins a 3 g s de força en cas el cas que la pista sigui horitzontal, això requereix molta força muscular.)
Angle inicial del pèndol/esquiador respecte de la vertical, Φ (º): 45
Percentatge de flexió i extensió de les cames (0-100%): 80
Mínim percentatge d'angulació (l'esquí assoleix 90º): 50.0 % (es recomana que sigui una mica més alt encara per evitar errors)
Màxim percentatge d'angulació (no hi ha angulació): 100 %
Percentatge d'angulació: 75


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:75: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.


#**Simulador de la trajectòria del pèndol centrífug invertit amb la respectiva trajectòria del centre de massa [Nivell: principiant-intermedi]**



Variables que et demanarà:


*   **Alçada de l'esquiador (m):** per calcular la posició aproximada del CM (centre de massa)
*   **Radi dels esquís (m)**
*   **Velocitat de l'esquiador durant el trajecte (m/s)(constant)**
*   **Temps final o durada del trajecte (s)**
*   **Angle inicial del pèndol/de l'esquiador (º):**

Hi han **dos** casos:

-Quan l'esquiador/el pèndol té 3 posicions d'equilibri. Dues de simètriques (t'ho indica abans que intodueixis l'angle) i una de vertical (0º). Òbviament, si aquest límit és sobrepassat l'esquiador perd l'equilibi i cau (en el cas del pèndol actua estranyament). Per això, diem que l'inclinació del pènol/de l'esquiador no pot passar del límit o angle màxim (99% de l'angle d'equilibri).

-Quan l'esquiador/el pèndol té una posició d'equilibri que és la vertical (0º) i, a més a més, si tries qualsevol angle, mentre sigui realista o no sigui extrem, pot estar oscil·lant constantment sense quedar-se en cap posició d'equilibri.

***Posició d'equilibri significa la posició de l'esquiador en què la velocitat angular del pèndol és 0.**


---
Normes per al correcte funcionament del simulador:

-Nombres decimals amb '  .  ' , no amb '  ,  '  .

**Correcte :** 12.7

**Incorrecte :** 12,7


---
Per iniciar la simulació cliqueu el botó de "play" al racó superior esquerre del codi o bé feu la combinació de tecles 

```
Ctrl+Enter
```




In [ ]:
import numpy as np
from numpy import sin,cos,sign,sqrt,arcsin,pi
from scipy.integrate import solve_ivp
from matplotlib import pyplot as plt
from termcolor import colored
from matplotlib.pyplot import figure
import sympy as smp
from matplotlib.animation import FuncAnimation, PillowWriter
from matplotlib import animation, rc
from IPython.display import HTML

print(colored("ADVERTÈNCIA: EL SIMULADOR ÉS VÀLID FINS APROXIMADAMENT 65º-70º","red"))
g = 9.807
height = float(input("Alçada de l'esquiador (m): "))
l = 0.56*height
R_sc = float(input("Radi dels esquís (m): "))
V = float(input("Velocitat de l'esquiador durant el trajecte (m/s): "))

if V**2/(g*R_sc)<1:
  Psi_eq = round(np.arcsin(V**2/(g*R_sc))*(180/np.pi), 3)
else:
  Psi_eq = "No hi han posicions d'equilibri"

if V**2/(g*R_sc)<1:
  text2 = "${\Psi}_{equilibri}$ = ±"
else:
  text2 = "${\Psi}_{equilibri}$ = "

def equations(t,y):
    dy=np.zeros([2])
    dy[0] = y[1]
    dy[1] = (g/l)*np.sin(y[0])-(V**2/(l*R_sc))*np.sign(y[0])

    return dy

final_time = int(input("Temps final o durada del trajecte (s): "))
interval=30*final_time
time = np.linspace(0, final_time, interval)

if V**2/(g*R_sc)<1:
  Psi_equilibri_arr = round(0.97*Psi_eq,3)
  print("Angle d'equilibri per referència: ±"+str(Psi_eq)+"º",  colored(("[NO POT PASSAR DE ± 0.99 · \u03A8  ≈ ± "+str(Psi_equilibri_arr)+"... º]"),"red"))
else:
  print("No hi han posicions d'equilibri, és a dir, el pèndol/l'esquiador oscilarà constantment a qualsevol angle sempre i quan no sigui extrem (70º-90º, 90º seria inclinar-se paral·lelament amb el terra | A partir de 70º l'esquiador pateix fins a 3 g s de força en cas el cas que la pista sigui horitzontal, això requereix molta força muscular.)")

initial_Psi = float(input("Angle inicial del pèndol/esquiador respecte de la vertical, \u03A6 (º): "))
y0 = [np.radians(initial_Psi), np.radians(0.0)]

res = solve_ivp(equations, (0,final_time), y0, method='RK45', t_eval=time, dense_output=True, rtol=1e-8, atol=1e-10)

t, psi, omega = res.t, res.y[0], res.y[1]

dense1 = res.sol
def dgamma(t, gamma):
    Psi, Omega = dense1(t)

    return [(V*np.sign(Psi))/(R_sc*np.cos(Psi))]

initial_gamma = [np.radians(0.0)]

gamma_res = solve_ivp(dgamma, (0,final_time), initial_gamma, method="RK45", t_eval = time, dense_output=True, rtol=1e-8, atol=1e-10)
gamma_times, gamma = gamma_res.t, gamma_res.y

dense2 = gamma_res.sol
def dx(t, x):
    Gamma = dense2(t)

    return [V*np.cos(Gamma)]

initial_x = [0.0]

x_res = solve_ivp(dx, (0,final_time), initial_x, method="RK45", t_eval = time, dense_output=True, rtol=1e-8, atol=1e-10)
x_times, x_skis = x_res.t, x_res.y

def dy(t, y):
    Gamma = dense2(t)

    return [V*np.sin(Gamma)]

initial_y = [0.0]

y_res = solve_ivp(dy, (0,final_time), initial_y, method="RK45", t_eval = time, dense_output=True, rtol=1e-8, atol=1e-10)
y_times, y_skis = y_res.t, y_res.y

if V**2/(g*R_sc)<1:
  text1 = '... º, ${\Psi}_{màx}$ = 0.99 · ${\Psi}_{equilibri}$'
else:
  text1 = "."

X_SKIS = x_skis.flatten()
Y_SKIS = y_skis.flatten()

var = 0
dist1 = []
length_array = 0
for i in range(1, len(X_SKIS)):
  length_array += np.sqrt((X_SKIS[i]-X_SKIS[i-1]) ** 2 + (Y_SKIS[i]-Y_SKIS[i-1]) ** 2 )
  dist1.append(length_array)
dist2 = dist1.insert(0,var)
xlim = V*np.cos(np.radians(initial_Psi))*final_time
ylim = 2*V*np.sin(np.radians(initial_Psi))

fig, ax = plt.subplots(figsize=(6.4*3.25,4.8*3.25))
line1, = ax.plot(0,0.1)  #, c="red", zorder=2, label="CM", edgecolors="black"
line2, = ax.plot(0,-0.1)  #, c="red", zorder=2, label="CM", edgecolors="black"
random = ax.plot(0,100, c="r",label="Traj. del CM")
trans = ax.transAxes
time_text2 = ax.text(0.005, 0.86, '', transform=trans, fontsize=8)
length_text  = ax.text(0.005, 0.92, '', transform=trans, fontsize=8)
ax.set_title('Trajectòria dels esquís i el centre de massa', fontsize=11,c="red")
ax.axis("scaled")
ax.set_ylabel("Y [m]", fontsize=9, fontweight = "bold")
ax.set_xlabel("X [m]", fontsize=9, fontweight = "bold")
plt.xticks(fontsize=7)
plt.yticks(fontsize=7)
ax.legend(loc='lower right',fontsize=7)
ax.xaxis.set_ticks(np.arange(0, 1.35*xlim, 5))
ax.set_xlim(0, 1.35*xlim)
ax.set_ylim(-3,ylim+3)
plt.close()

x_anim = []
y1_anim = []
y2_anim = []
y3_anim = []

def animate(i):
  x_anim.append(x_skis.T[i])
  y1_anim.append(y_skis.T[i]+0.1)
  y2_anim.append(y_skis.T[i]-0.1)
  CM_Y = l*np.sin(psi[i])
  y3_anim.append(y_skis.T[i]+CM_Y)
  ax.plot(x_anim, y1_anim,"k", linewidth=0.25)
  ax.plot(x_anim, y2_anim,"k", linewidth=0.25)
  ax.plot(x_anim, y3_anim,"r", linewidth=0.25)
  time_text2.set_text('Temps = %.1f s' %(i/30))
  length_text.set_text('Distància recorreguda = %.2f m' %(dist1[i]))
  return line1, line2, time_text2, length_text

anim = FuncAnimation(fig, animate, frames=interval, interval=33.3333334, blit=True)

HTML(anim.to_html5_video())

ADVERTÈNCIA: EL SIMULADOR ÉS VÀLID FINS APROXIMADAMENT 65º-70º
Alçada de l'esquiador (m): 1.8
Radi dels esquís (m): 14
Velocitat de l'esquiador durant el trajecte (m/s): 12
Temps final o durada del trajecte (s): 10
No hi han posicions d'equilibri, és a dir, el pèndol/l'esquiador oscilarà constantment a qualsevol angle sempre i quan no sigui extrem (70º-90º, 90º seria inclinar-se paral·lelament amb el terra | A partir de 70º l'esquiador pateix fins a 3 g s de força en cas el cas que la pista sigui horitzontal, això requereix molta força muscular.)
Angle inicial del pèndol/esquiador respecte de la vertical, Φ (º): 45


#**Simulador de la trajectòria del pèndol centrífug invertit amb la respectiva trajectòria del centre de massa (amb flexió i extensió de cames afegida) [Nivell: experimentat]**



Variables que et demanarà:

*   **Alçada de l'esquiador (m):** per calcular la posició aproximada del CM (centre de massa)
*   **Radi dels esquís (m)**
*   **Velocitat de l'esquiador durant el trajecte (m/s)(constant)**
*   **Temps final o durada del trajecte (s)**
*   **Angle inicial del pèndol/de l'esquiador (º):** 

Hi han **dos** casos:

-Quan l'esquiador/el pèndol té 3 posicions d'equilibri. Dues de simètriques (t'ho indica abans que intodueixis l'angle) i una de vertical (0º). Òbviament, si aquest límit és sobrepassat l'esquiador perd l'equilibi i cau (en el cas del pèndol actua estranyament). Per això, diem que l'inclinació del pènol/de l'esquiador no pot passar del límit o angle màxim (99% de l'angle d'equilibri).

-Quan l'esquiador/el pèndol té una posició d'equilibri que és la vertical (0º) i, a més a més, si tries qualsevol angle, mentre sigui realista o no sigui extrem, pot estar oscil·lant constantment sense quedar-se en cap posició d'equilibri.

***Posició d'equilibri significa la posició de l'esquiador en què la velocitat angular del pèndol és 0.**

*   **Percentatge de flexió i extensió de les cames:** quan l'esquiador està en posició vertical les cames estan flexionades, per altra banda, si està al extrem de la curva la cama exterior està extesa (l'altra és paral·lela a l'exterior i, per tant, sempre està flexionada, vertical o no vertical). Generalment, com més petit és el percentatge, menys es nota la flexió i extensió de cames, i, a l'inrevés, com més s'aproxima al 100% més es nota la flexió i extensió de cames.



---
Normes per al correcte funcionament del simulador:

-Nombres decimals amb '  .  ' , no amb '  ,  '  .

**Correcte :** 12.7

**Incorrecte :** 12,7


---
Per iniciar la simulació cliqueu el botó de "play" al racó superior esquerre del codi o bé feu la combinació de tecles

```
Ctrl+Enter
```




In [4]:
import numpy as np
from numpy import sin,cos,sign,sqrt,arcsin,pi
from scipy.integrate import solve_ivp
from matplotlib import pyplot as plt
from termcolor import colored
from matplotlib.pyplot import figure
import sympy as smp
from matplotlib.animation import FuncAnimation, PillowWriter
from matplotlib import animation, rc
from IPython.display import HTML

print(colored("ADVERTÈNCIA: EL SIMULADOR ÉS VÀLID FINS APROXIMADAMENT 65º-70º","red"))
g = 9.807
height = float(input("Alçada de l'esquiador (m): "))
l_o = 0.56*height
R_sc = float(input("Radi dels esquís (m): "))
V = float(input("Velocitat de l'esquiador durant el trajecte (m/s): "))

if V**2/(g*R_sc)<1:
  Psi_eq = round(np.arcsin(V**2/(g*R_sc))*(180/np.pi), 3)
else:
  Psi_eq = "No hi han posicions d'equilibri"

if V**2/(g*R_sc)<1:
  text2 = "${\Psi}_{equilibri}$ = ±"
else:
  text2 = "${\Psi}_{equilibri}$ = "

def equations(t,y):
    dy=np.zeros([2])
    dy[0] = y[1]
    dy[1] = (g/(l_o*(1-b*np.cos(y[0]))**(1./3)))*np.sin(y[0])-((V**2)/(R_sc*l_o*(1-b*np.cos(y[0]))**(1./3)))*np.sign(y[0]) - (2 * b * (y[1])**2 * np.sin(y[0]))/(3-3 * b * np.cos(y[0]))

    return dy

final_time = int(input("Temps final o durada del trajecte (s): "))
interval=30*final_time
time = np.linspace(0, final_time, interval)

if V**2/(g*R_sc)<1:
  Psi_equilibri_arr = round(0.97*Psi_eq,3)
  print("Angle d'equilibri per referència: ±"+str(Psi_eq)+"º",  colored(("[NO POT PASSAR DE ± 0.99 · \u03A8  ≈ ± "+str(Psi_equilibri_arr)+"... º]"),"red"))
else:
  print("No hi han posicions d'equilibri, és a dir, el pèndol/l'esquiador oscilarà constantment a qualsevol angle sempre i quan no sigui extrem (70º-90º, 90º seria inclinar-se paral·lelament amb el terra | A partir de 70º l'esquiador pateix fins a 3 g s de força en cas el cas que la pista sigui horitzontal, això requereix molta força muscular.)")

initial_Psi = float(input("Angle inicial del pèndol/esquiador respecte de la vertical, \u03A6 (º): "))
P = float(input("Percentatge de flexió i extensió de les cames (0-100%): "))
b = P/100
y0 = [np.radians(initial_Psi), np.radians(0.0)]

res = solve_ivp(equations, (0,final_time), y0, method='RK45', t_eval=time, dense_output=True, rtol=1e-8, atol=1e-10)

t, psi, omega = res.t, res.y[0], res.y[1]

dense1 = res.sol
def dgamma(t, gamma):
    Psi, Omega = dense1(t)

    return [(V*np.sign(Psi))/(R_sc*np.cos(Psi))]

initial_gamma = [np.radians(0.0)]

gamma_res = solve_ivp(dgamma, (0,final_time), initial_gamma, method="RK45", t_eval = time, dense_output=True, rtol=1e-8, atol=1e-10)
gamma_times, gamma = gamma_res.t, gamma_res.y

dense2 = gamma_res.sol
def dx(t, x):
    Gamma = dense2(t)

    return [V*np.cos(Gamma)]

initial_x = [0.0]

x_res = solve_ivp(dx, (0,final_time), initial_x, method="RK45", t_eval = time, dense_output=True, rtol=1e-8, atol=1e-10)
x_times, x_skis = x_res.t, x_res.y

def dy(t, y):
    Gamma = dense2(t)

    return [V*np.sin(Gamma)]

initial_y = [0.0]

y_res = solve_ivp(dy, (0,final_time), initial_y, method="RK45", t_eval = time, dense_output=True, rtol=1e-8, atol=1e-10)
y_times, y_skis = y_res.t, y_res.y

if V**2/(g*R_sc)<1:
  text1 = '... º, ${\Psi}_{màx}$ = 0.99 · ${\Psi}_{equilibri}$'
else:
  text1 = "."

X_SKIS = x_skis.flatten()
Y_SKIS = y_skis.flatten()

var = 0
dist1 = []
length_array = 0
for i in range(1, len(X_SKIS)):
  length_array += np.sqrt((X_SKIS[i]-X_SKIS[i-1]) ** 2 + (Y_SKIS[i]-Y_SKIS[i-1]) ** 2 )
  dist1.append(length_array)
dist2 = dist1.insert(0,var)
xlim = V*np.cos(np.radians(initial_Psi))*final_time
ylim = 2*V*np.sin(np.radians(initial_Psi))

fig, ax = plt.subplots(figsize=(6.4*3.25,4.8*3.25))
line1, = ax.plot(0,0.1)  #, c="red", zorder=2, label="CM", edgecolors="black"
line2, = ax.plot(0,-0.1)  #, c="red", zorder=2, label="CM", edgecolors="black"
random = ax.plot(0,100, c="r",label="Traj. del CM")
trans = ax.transAxes
time_text2 = ax.text(0.005, 0.86, '', transform=trans, fontsize=8)
length_text  = ax.text(0.005, 0.92, '', transform=trans, fontsize=8)
ax.set_title('Trajectòria dels esquís i el centre de massa', fontsize=11,c="red")
ax.axis("scaled")
ax.set_ylabel("Y [m]", fontsize=9, fontweight = "bold")
ax.set_xlabel("X [m]", fontsize=9, fontweight = "bold")
plt.xticks(fontsize=7)
plt.yticks(fontsize=7)
ax.legend(loc='lower right',fontsize=7)
ax.xaxis.set_ticks(np.arange(0, 1.35*xlim, 5))
ax.set_xlim(0, 1.35*xlim)
ax.set_ylim(-3,ylim+3)
plt.close()

x_anim = []
y1_anim = []
y2_anim = []
y3_anim = []

def animate(i):
  x_anim.append(x_skis.T[i])
  y1_anim.append(y_skis.T[i]+0.1)
  y2_anim.append(y_skis.T[i]-0.1)
  CM_Y = l_o * np.sin(psi[i]) * (1-b*np.cos(psi[i]))**(1./3)
  y3_anim.append(y_skis.T[i]+CM_Y)
  ax.plot(x_anim, y1_anim,"k", linewidth=0.25)
  ax.plot(x_anim, y2_anim,"k", linewidth=0.25)
  ax.plot(x_anim, y3_anim,"r", linewidth=0.25)
  time_text2.set_text('Temps = %.1f s' %(i/30))
  length_text.set_text('Distància recorreguda = %.2f m' %(dist1[i]))
  return line1, line2, time_text2, length_text

anim = FuncAnimation(fig, animate, frames=interval, interval=33.3333334, blit=True)

HTML(anim.to_html5_video())

ADVERTÈNCIA: EL SIMULADOR ÉS VÀLID FINS APROXIMADAMENT 65º-70º
Alçada de l'esquiador (m): 1.8
Radi dels esquís (m): 14
Velocitat de l'esquiador durant el trajecte (m/s): 12
Temps final o durada del trajecte (s): 10
No hi han posicions d'equilibri, és a dir, el pèndol/l'esquiador oscilarà constantment a qualsevol angle sempre i quan no sigui extrem (70º-90º, 90º seria inclinar-se paral·lelament amb el terra | A partir de 70º l'esquiador pateix fins a 3 g s de força en cas el cas que la pista sigui horitzontal, això requereix molta força muscular.)
Angle inicial del pèndol/esquiador respecte de la vertical, Φ (º): 45
Percentatge de flexió i extensió de les cames (0-100%): 80


#**Simulador de la trajectòria del pèndol centrífug invertit amb la respectiva trajectòria del centre de massa (amb angulació afegida) [Nivell: expert]**



Variables que et demanarà:

*   **Alçada de l'esquiador (m):** per calcular la posició aproximada del CM (centre de massa)
*   **Radi dels esquís (m)**
*  **Velocitat de l'esquiador durant el trajecte (m/s)(constant)**
*   **Temps final o durada del trajecte (s)**
*  **Angle inicial del pèndol/de l'esquiador (º):**

Hi han **dos** casos:

-Quan l'esquiador/el pèndol té 3 posicions d'equilibri. Dues de simètriques (t'ho indica abans que intodueixis l'angle) i una de vertical (0º). Òbviament, si aquest límit és sobrepassat l'esquiador perd l'equilibi i cau (en el cas del pèndol actua estranyament). Per això, diem que l'inclinació del pènol/de l'esquiador no pot passar del límit o angle màxim (99% de l'angle d'equilibri).

-Quan l'esquiador/el pèndol té una posició d'equilibri que és la vertical (0º) i, a més a més, si tries qualsevol angle, mentre sigui realista o no sigui extrem, pot estar oscil·lant constantment sense quedar-se en cap posició d'equilibri.

***Posició d'equilibri significa la posició de l'esquiador en què la velocitat angular del pèndol és 0.**

* **Percentatge d'angulació (Angle d'amplitud del pèndol = Percentatge * Angle de l'esquí respecte del terra):** l'angle de l'esquí sempre serà més gran que la de l'esquiador. Està comprès entre 0-100%. A 100% no hi ha angulació, atès que és com si multipliquessis per 1.)


---
Normes per al correcte funcionament del simulador:

-Nombres decimals amb '  .  ' , no amb '  ,  '  .

**Correcte :** 12.7

**Incorrecte :** 12,7


---
Per iniciar la simulació cliqueu el botó de "play" al racó superior esquerre del codi o bé feu la combinació de tecles

```
Ctrl+Enter
```

In [ ]:
import numpy as np
from numpy import sin,cos,sign,sqrt,arcsin,pi
from scipy.integrate import solve_ivp
from matplotlib import pyplot as plt
from termcolor import colored
from matplotlib.pyplot import figure
import sympy as smp
from matplotlib.animation import FuncAnimation, PillowWriter
from matplotlib import animation, rc
from IPython.display import HTML

print(colored("ADVERTÈNCIA: EL SIMULADOR ÉS VÀLID FINS APROXIMADAMENT 65º-70º","red"))
g = 9.807
height = float(input("Alçada de l'esquiador (m): "))
l = 0.56*height
R_sc = float(input("Radi dels esquís (m): "))
V = float(input("Velocitat de l'esquiador durant el trajecte (m/s): "))

if V**2/(g*R_sc)<1:
  Psi_eq = round(np.arcsin(V**2/(g*R_sc))*(180/np.pi), 3)
else:
  Psi_eq = "No hi han posicions d'equilibri"

if V**2/(g*R_sc)<1:
  text2 = "${\Psi}_{equilibri}$ = ±"
else:
  text2 = "${\Psi}_{equilibri}$ = "

def equations(t,y):
    dy=np.zeros([2])
    dy[0] = y[1]
    dy[1] = (g/l*A)*np.sin(A*y[0])-(V**2 * np.cos(A*y[0]) * np.sign(y[0]))/(l*R_sc*A*np.cos(y[0]))

    return dy

final_time = int(input("Temps final o durada del trajecte (s): "))
interval=30*final_time
time = np.linspace(0, final_time, interval)

if V**2/(g*R_sc)<1:
  Psi_equilibri_arr = round(0.97*Psi_eq,3)
  print("Angle d'equilibri per referència: ±"+str(Psi_eq)+"º",  colored(("[NO POT PASSAR DE ± 0.99 · \u03A8  ≈ ± "+str(Psi_equilibri_arr)+"... º]"),"red"))
else:
  print("No hi han posicions d'equilibri, és a dir, el pèndol/l'esquiador oscilarà constantment a qualsevol angle sempre i quan no sigui extrem (70º-90º, 90º seria inclinar-se paral·lelament amb el terra | A partir de 70º l'esquiador pateix fins a 3 g s de força en cas el cas que la pista sigui horitzontal, això requereix molta força muscular.)")

initial_Psi = float(input("Angle inicial del pèndol/esquiador respecte de la vertical, \u03A6 (º): "))
print("Mínim percentatge d'angulació (l'esquí assoleix 90º): "+str((initial_Psi*100)/90)+" % (es recomana que sigui una mica més alt encara per evitar errors)")
print("Màxim percentatge d'angulació (no hi ha angulació): 100 %")
P = float(input("Percentatge d'angulació: "))
A = P/100
y0 = [np.radians(initial_Psi/A), np.radians(0.0)]


res = solve_ivp(equations, (0,final_time), y0, method='RK45', t_eval=time, dense_output=True, rtol=1e-8, atol=1e-10)

t, psi, omega = res.t, res.y[0], res.y[1]

dense1 = res.sol
def dgamma(t, gamma):
    Psi, Omega = dense1(t)

    return [(V*np.sign(Psi))/(R_sc*np.cos(Psi))]

initial_gamma = [np.radians(0.0)]

gamma_res = solve_ivp(dgamma, (0,final_time), initial_gamma, method="RK45", t_eval = time, dense_output=True, rtol=1e-8, atol=1e-10)
gamma_times, gamma = gamma_res.t, gamma_res.y

dense2 = gamma_res.sol
def dx(t, x):
    Gamma = dense2(t)

    return [V*np.cos(Gamma)]

initial_x = [0.0]

x_res = solve_ivp(dx, (0,final_time), initial_x, method="RK45", t_eval = time, dense_output=True, rtol=1e-8, atol=1e-10)
x_times, x_skis = x_res.t, x_res.y

def dy(t, y):
    Gamma = dense2(t)

    return [V*np.sin(Gamma)]

initial_y = [0.0]

y_res = solve_ivp(dy, (0,final_time), initial_y, method="RK45", t_eval = time, dense_output=True, rtol=1e-8, atol=1e-10)
y_times, y_skis = y_res.t, y_res.y

if V**2/(g*R_sc)<1:
  text1 = '... º, ${\Psi}_{màx}$ = 0.99 · ${\Psi}_{equilibri}$'
else:
  text1 = "."

X_SKIS = x_skis.flatten()
Y_SKIS = y_skis.flatten()

var = 0
dist1 = []
length_array = 0
for i in range(1, len(X_SKIS)):
  length_array += np.sqrt((X_SKIS[i]-X_SKIS[i-1]) ** 2 + (Y_SKIS[i]-Y_SKIS[i-1]) ** 2 )
  dist1.append(length_array)
dist2 = dist1.insert(0,var)
xlim = V*np.cos(np.radians(initial_Psi))*final_time
ylim = 2*V*np.sin(np.radians(initial_Psi))

fig, ax = plt.subplots(figsize=(6.4*3.25,4.8*3.25))
line1, = ax.plot(0,0.1)  #, c="red", zorder=2, label="CM", edgecolors="black"
line2, = ax.plot(0,-0.1)  #, c="red", zorder=2, label="CM", edgecolors="black"
random = ax.plot(0,100, c="r",label="Traj. del CM")
trans = ax.transAxes
time_text2 = ax.text(0.005, 0.86, '', transform=trans, fontsize=8)
length_text  = ax.text(0.005, 0.92, '', transform=trans, fontsize=8)
ax.set_title('Trajectòria dels esquís i el centre de massa', fontsize=11,c="red")
ax.axis("scaled")
ax.set_ylabel("Y [m]", fontsize=9, fontweight = "bold")
ax.set_xlabel("X [m]", fontsize=9, fontweight = "bold")
plt.xticks(fontsize=7)
plt.yticks(fontsize=7)
ax.legend(loc='lower right',fontsize=7)
ax.xaxis.set_ticks(np.arange(0, 1.35*xlim, 5))
ax.set_xlim(0, 1.35*xlim)
ax.set_ylim(-3,ylim-3)
plt.close()

x_anim = []
y1_anim = []
y2_anim = []
y3_anim = []

def animate(i):
  x_anim.append(x_skis.T[i])
  y1_anim.append(y_skis.T[i]+0.1)
  y2_anim.append(y_skis.T[i]-0.1)
  CM_Y = l*np.sin(A*psi[i])
  y3_anim.append(y_skis.T[i]+CM_Y)
  ax.plot(x_anim, y1_anim,"k", linewidth=0.25)
  ax.plot(x_anim, y2_anim,"k", linewidth=0.25)
  ax.plot(x_anim, y3_anim,"r", linewidth=0.25)
  time_text2.set_text('Temps = %.1f s' %(i/30))
  length_text.set_text('Distància recorreguda = %.2f m' %(dist1[i]))
  return line1, line2, time_text2, length_text

anim = FuncAnimation(fig, animate, frames=interval, interval=33.3333334, blit=True)

HTML(anim.to_html5_video())

ADVERTÈNCIA: EL SIMULADOR ÉS VÀLID FINS APROXIMADAMENT 65º-70º
Alçada de l'esquiador (m): 1.8
Radi dels esquís (m): 14
Velocitat de l'esquiador durant el trajecte (m/s): 12
Temps final o durada del trajecte (s): 10
No hi han posicions d'equilibri, és a dir, el pèndol/l'esquiador oscilarà constantment a qualsevol angle sempre i quan no sigui extrem (70º-90º, 90º seria inclinar-se paral·lelament amb el terra | A partir de 70º l'esquiador pateix fins a 3 g s de força en cas el cas que la pista sigui horitzontal, això requereix molta força muscular.)
Angle inicial del pèndol/esquiador respecte de la vertical, Φ (º): 45
Mínim percentatge d'angulació (l'esquí assoleix 90º): 50.0 % (es recomana que sigui una mica més alt encara per evitar errors)
Màxim percentatge d'angulació (no hi ha angulació): 100 %
Percentatge d'angulació: 55


#**Simulador de la trajectòria del pèndol centrífug invertit amb la respectiva trajectòria del centre de massa (amb flexió i extensió de cames i amb angulació afegida) [Nivell: atleta-professional]**



Variables que et demanarà:

*   **Alçada de l'esquiador (m):** per calcular la posició aproximada del CM (centre de massa)
*   **Radi dels esquís (m)**
*  **Velocitat de l'esquiador durant el trajecte (m/s)(constant)**
*   **Temps final o durada del trajecte (s)**
*  **Angle inicial del pèndol/de l'esquiador (º):**

Hi han **dos** casos:

-Quan l'esquiador/el pèndol té 3 posicions d'equilibri. Dues de simètriques (t'ho indica abans que intodueixis l'angle) i una de vertical (0º). Òbviament, si aquest límit és sobrepassat l'esquiador perd l'equilibi i cau (en el cas del pèndol actua estranyament). Per això, diem que l'inclinació del pènol/de l'esquiador no pot passar del límit o angle màxim (99% de l'angle d'equilibri).

-Quan l'esquiador/el pèndol té una posició d'equilibri que és la vertical (0º) i, a més a més, si tries qualsevol angle, mentre sigui realista o no sigui extrem, pot estar oscil·lant constantment sense quedar-se en cap posició d'equilibri.

***Posició d'equilibri significa la posició de l'esquiador en què la velocitat angular del pèndol és 0.**



*   **Percentatge de flexió i extensió de les cames:** quan l'esquiador està en posició vertical les cames estan flexionades, per altra banda, si està al extrem de la curva la cama exterior està extesa (l'altra és paral·lela a l'exterior i, per tant, sempre està flexionada, vertical o no vertical). Generalment, com més petit és el percentatge, menys es nota la flexió i extensió de cames, i, a l'inrevés, com més s'aproxima al 100% més es nota la flexió i extensió de cames.


* **Percentatge d'angulació (Angle d'amplitud del pèndol = Percentatge * Angle de l'esquí respecte del terra):** l'angle de l'esquí sempre serà més gran que la de l'esquiador. Està comprès entre 0-100%. A 100% no hi ha angulació, atès que és com si multipliquessis per 1.)

---
Normes per al correcte funcionament del simulador:

-Nombres decimals amb '  .  ' , no amb '  ,  '  .

**Correcte :** 12.7

**Incorrecte :** 12,7


---
Per iniciar la simulació cliqueu el botó de "play" al racó superior esquerre del codi o bé feu la combinació de tecles

```
Ctrl+Enter
```

In [10]:
import numpy as np
from numpy import sin,cos,sign,sqrt,arcsin,pi
from scipy.integrate import solve_ivp
from matplotlib import pyplot as plt
from termcolor import colored
from matplotlib.pyplot import figure
import sympy as smp
from matplotlib.animation import FuncAnimation, PillowWriter
from matplotlib import animation, rc
from IPython.display import HTML

print(colored("ADVERTÈNCIA: EL SIMULADOR ÉS VÀLID FINS APROXIMADAMENT 65º-70º","red"))
g = 9.807
height = float(input("Alçada de l'esquiador (m): "))
l = 0.56*height
R_sc = float(input("Radi dels esquís (m): "))
V = float(input("Velocitat de l'esquiador durant el trajecte (m/s): "))

if V**2/(g*R_sc)<1:
  Psi_eq = round(np.arcsin(V**2/(g*R_sc))*(180/np.pi), 3)
else:
  Psi_eq = "No hi han posicions d'equilibri"

if V**2/(g*R_sc)<1:
  text2 = "${\Psi}_{equilibri}$ = ±"
else:
  text2 = "${\Psi}_{equilibri}$ = "

def equations(t,y):
    dy=np.zeros([2])
    dy[0] = y[1]
    dy[1] = (g/(A * l_o * (1-b * np.cos(A*y[0]))**(1./3)))*np.sin(A*y[0])-(V**2 * np.cos(A*y[0]) * np.sign(y[0]))/(l_o * (1-b * np.cos(A*y[0]))**(1./3) * R_sc * A * np.cos(y[0])) - (2 * b * (y[1])**2 * np.sin(A*y[0]))/(3*A-3* A * b * np.cos(A*y[0]))

    return dy

final_time = int(input("Temps final o durada del trajecte (s): "))
interval=30*final_time
time = np.linspace(0, final_time, interval)

if V**2/(g*R_sc)<1:
  Psi_equilibri_arr = round(0.97*Psi_eq,3)
  print("Angle d'equilibri per referència: ±"+str(Psi_eq)+"º",  colored(("[NO POT PASSAR DE ± 0.99 · \u03A8  ≈ ± "+str(Psi_equilibri_arr)+"... º]"),"red"))
else:
  print("No hi han posicions d'equilibri, és a dir, el pèndol/l'esquiador oscilarà constantment a qualsevol angle sempre i quan no sigui extrem (70º-90º, 90º seria inclinar-se paral·lelament amb el terra | A partir de 70º l'esquiador pateix fins a 3 g s de força en cas el cas que la pista sigui horitzontal, això requereix molta força muscular.)")

initial_Psi = float(input("Angle inicial del pèndol/esquiador respecte de la vertical, \u03A6 (º): "))
P_1 = float(input("Percentatge de flexió i extensió de les cames (0-100%): "))
b = P_1/100
print("Mínim percentatge d'angulació (l'esquí assoleix 90º): "+str((initial_Psi*100)/90)+" % (es recomana que sigui una mica més alt encara per evitar errors)")
print("Màxim percentatge d'angulació (no hi ha angulació): 100 %")
P_2 = float(input("Percentatge d'angulació: "))
A = P_2/100
y0 = [np.radians(initial_Psi/A), np.radians(0.0)]


res = solve_ivp(equations, (0,final_time), y0, method='RK45', t_eval=time, dense_output=True, rtol=1e-8, atol=1e-10)

t, psi, omega = res.t, res.y[0], res.y[1]

dense1 = res.sol
def dgamma(t, gamma):
    Psi, Omega = dense1(t)

    return [(V*np.sign(Psi))/(R_sc*np.cos(Psi))]

initial_gamma = [np.radians(0.0)]

gamma_res = solve_ivp(dgamma, (0,final_time), initial_gamma, method="RK45", t_eval = time, dense_output=True, rtol=1e-8, atol=1e-10)
gamma_times, gamma = gamma_res.t, gamma_res.y

dense2 = gamma_res.sol
def dx(t, x):
    Gamma = dense2(t)

    return [V*np.cos(Gamma)]

initial_x = [0.0]

x_res = solve_ivp(dx, (0,final_time), initial_x, method="RK45", t_eval = time, dense_output=True, rtol=1e-8, atol=1e-10)
x_times, x_skis = x_res.t, x_res.y

def dy(t, y):
    Gamma = dense2(t)

    return [V*np.sin(Gamma)]

initial_y = [0.0]

y_res = solve_ivp(dy, (0,final_time), initial_y, method="RK45", t_eval = time, dense_output=True, rtol=1e-8, atol=1e-10)
y_times, y_skis = y_res.t, y_res.y

if V**2/(g*R_sc)<1:
  text1 = '... º, ${\Psi}_{màx}$ = 0.99 · ${\Psi}_{equilibri}$'
else:
  text1 = "."

X_SKIS = x_skis.flatten()
Y_SKIS = y_skis.flatten()

var = 0
dist1 = []
length_array = 0
for i in range(1, len(X_SKIS)):
  length_array += np.sqrt((X_SKIS[i]-X_SKIS[i-1]) ** 2 + (Y_SKIS[i]-Y_SKIS[i-1]) ** 2 )
  dist1.append(length_array)
dist2 = dist1.insert(0,var)
xlim = V*np.cos(np.radians(initial_Psi))*final_time
ylim = 2*V*np.sin(np.radians(initial_Psi))

fig, ax = plt.subplots(figsize=(6.4*3.25,4.8*3.25))
line1, = ax.plot(0,0.1)  #, c="red", zorder=2, label="CM", edgecolors="black"
line2, = ax.plot(0,-0.1)  #, c="red", zorder=2, label="CM", edgecolors="black"
random = ax.plot(0,100, c="r",label="Traj. del CM")
trans = ax.transAxes
time_text2 = ax.text(0.005, 0.86, '', transform=trans, fontsize=8)
length_text  = ax.text(0.005, 0.92, '', transform=trans, fontsize=8)
ax.set_title('Trajectòria dels esquís i el centre de massa', fontsize=11,c="red")
ax.axis("scaled")
ax.set_ylabel("Y [m]", fontsize=9, fontweight = "bold")
ax.set_xlabel("X [m]", fontsize=9, fontweight = "bold")
plt.xticks(fontsize=7)
plt.yticks(fontsize=7)
ax.legend(loc='lower right',fontsize=7)
ax.xaxis.set_ticks(np.arange(0, 1.35*xlim, 5))
ax.set_xlim(0, 1.35*xlim)
ax.set_ylim(-3,ylim-3)
plt.close()

x_anim = []
y1_anim = []
y2_anim = []
y3_anim = []

def animate(i):
  x_anim.append(x_skis.T[i])
  y1_anim.append(y_skis.T[i]+0.1)
  y2_anim.append(y_skis.T[i]-0.1)
  CM_Y = l*np.sin(A*psi[i])
  y3_anim.append(y_skis.T[i]+CM_Y)
  ax.plot(x_anim, y1_anim,"k", linewidth=0.25)
  ax.plot(x_anim, y2_anim,"k", linewidth=0.25)
  ax.plot(x_anim, y3_anim,"r", linewidth=0.25)
  time_text2.set_text('Temps = %.1f s' %(i/30))
  length_text.set_text('Distància recorreguda = %.2f m' %(dist1[i]))
  return line1, line2, time_text2, length_text

anim = FuncAnimation(fig, animate, frames=interval, interval=33.3333334, blit=True)

HTML(anim.to_html5_video())

ADVERTÈNCIA: EL SIMULADOR ÉS VÀLID FINS APROXIMADAMENT 65º-70º
Alçada de l'esquiador (m): 1.8
Radi dels esquís (m): 14
Velocitat de l'esquiador durant el trajecte (m/s): 12
Temps final o durada del trajecte (s): 10
No hi han posicions d'equilibri, és a dir, el pèndol/l'esquiador oscilarà constantment a qualsevol angle sempre i quan no sigui extrem (70º-90º, 90º seria inclinar-se paral·lelament amb el terra | A partir de 70º l'esquiador pateix fins a 3 g s de força en cas el cas que la pista sigui horitzontal, això requereix molta força muscular.)
Angle inicial del pèndol/esquiador respecte de la vertical, Φ (º): 45
Percentatge de flexió i extensió de les cames (0-100%): 80
Mínim percentatge d'angulació (l'esquí assoleix 90º): 50.0 % (es recomana que sigui una mica més alt encara per evitar errors)
Màxim percentatge d'angulació (no hi ha angulació): 100 %
Percentatge d'angulació: 75
